In [45]:
import os

In [46]:
os.chdir('C:/Users/ba415/OneDrive/Desktop/ENDTOEND-REALESTATE-MLPROJECT')
print(os.getcwd())  # Ensure it prints the project root directory

C:\Users\ba415\OneDrive\Desktop\ENDTOEND-REALESTATE-MLPROJECT


In [47]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainingConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    learning_rate: float
    max_depth: int
    min_samples_leaf: int
    min_samples_split: int
    n_estimators: int
    target_column: str

In [48]:
from src.mlProject.constants import *
from src.mlProject.utils.common import read_yaml, create_directories
from box import ConfigBox

In [49]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath: Path = Path(CONFIG_FILE_PATH),
        params_filepath: Path = Path(PARAMS_FILE_PATH),
        schema_filepath: Path = Path(SCHEMA_FILE_PATH)
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainingConfig:
        config = self.config.model_trainer
        params = self.params.GradientBoostingRegressor

        # Ensure schema is accessed correctly
        schema = self.schema.to_dict() if isinstance(self.schema, ConfigBox) else self.schema
        target_column = schema.get('TARGET_COLUMN', {}).get('name', 'default_target_column')

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainingConfig(
            root_dir=config.root_dir,
            train_data_path=config.train_data_path,
            test_data_path=config.test_data_path,
            model_name=config.model_name,
            learning_rate=params.learning_rate,
            max_depth=params.max_depth,
            min_samples_leaf=params.min_samples_leaf,
            min_samples_split=params.min_samples_split,
            n_estimators=params.n_estimators,
            target_column=target_column
        )

        return model_trainer_config

In [50]:
import pandas as pd
import os
from src.mlProject import logger
from sklearn.ensemble import GradientBoostingRegressor
import joblib

In [51]:
class ModelTrainer:
    def __init__(self, config: ModelTrainingConfig):
        self.config = config

    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        train_x = train_data.drop(columns=[self.config.target_column])
        test_x = test_data.drop(columns=[self.config.target_column])
        train_y = train_data[self.config.target_column]
        test_y = test_data[self.config.target_column]

        gbr = GradientBoostingRegressor(
            learning_rate=self.config.learning_rate, 
            max_depth=self.config.max_depth,
            min_samples_leaf=self.config.min_samples_leaf,
            min_samples_split=self.config.min_samples_split,
            n_estimators=self.config.n_estimators
        )
        gbr.fit(train_x, train_y)

        model_path = os.path.join(self.config.root_dir, self.config.model_name)
        joblib.dump(gbr, model_path)
        logger.info(f"Model saved at {model_path}")

In [52]:
# Main script
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(config=model_trainer_config)
    model_trainer.train()
except Exception as e:
    logger.error(f"Error in model training: {e}")
    raise e

[2024-08-01 01:26:24,736: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-08-01 01:26:24,739: INFO: common: yaml file: params.yaml loaded successfully]
[2024-08-01 01:26:24,742: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-08-01 01:26:24,743: INFO: common: created directory at: artifacts]
[2024-08-01 01:26:24,745: INFO: common: created directory at: artifacts/model_training]
[2024-08-01 01:26:24,994: INFO: 3966667851: Model saved at artifacts/model_training\model.joblib]


In [ ]:
# model training done